### DEV


In [ ]:
from pathlib import Path
import sys
import os
import django
import pandas as pd
from asgiref.sync import sync_to_async

# Add the parent directory to the system path and setup django
BASE_DIR = str(Path(os.getcwd()).resolve().parent.parent)

if BASE_DIR not in sys.path:
    sys.path.append(BASE_DIR)

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "backend_site.settings")

django.setup()

from tsosi.models import Entity, Transfer


@sync_to_async
def run():

    # data = Transfer.objects.filter(emitter__country__isnull=False).values(
    #     "emitter__id", "emitter__country"
    # )
    data = Entity.objects.filter(
        country__isnull=False, is_recipient=False
    ).values("id", "country")
    df = pd.DataFrame.from_records(data)

    return df


res = await run()

In [4]:
res

,emitter__id,emitter__country
0,000c71e1-ec1e-475d-97c2-7791e0a4f70c,FR
1,000c71e1-ec1e-475d-97c2-7791e0a4f70c,FR
2,000c71e1-ec1e-475d-97c2-7791e0a4f70c,FR
3,000c71e1-ec1e-475d-97c2-7791e0a4f70c,FR
4,000c71e1-ec1e-475d-97c2-7791e0a4f70c,FR
...,...,...
2713,ffae2bf9-de2d-44eb-858b-21557e527540,CH
2714,ffae2bf9-de2d-44eb-858b-21557e527540,CH
2715,ffae2bf9-de2d-44eb-858b-21557e527540,CH
2716,ffae2bf9-de2d-44eb-858b-21557e527540,CH


In [ ]:
from pycountry import countries

d = (
    res.groupby("country")
    .count()
    .reset_index()
    .sort_values("id", ascending=False)
)
mapping = {c.alpha_2: c.name for c in countries}
d["country_name"] = d["country"].map(mapping)
d.rename(columns={"country": "country_code", "id": "number"}, inplace=True)


categories = [(50, "Upper"), (10, "Medium"), (0, "Lower")]

for c_threshold, c_name in sorted(categories, key=lambda x: x[0]):
    mask = d["number"] >= c_threshold
    d.loc[mask, "category"] = c_name

d[["country_name", "country_code", "number", "category"]].to_excel(
    "supporters_extract.xlsx", index=False
)

In [ ]:
sorted(categories, key=lambda x: x[0])

In [ ]:
from fontTools.ttLib.woff2 import compress
from pathlib import Path
import re

font_directory = Path.home() / "Documents/TSOSI/Graphisme/Mont/Mont"
otf_files = font_directory.glob("*.otf")
save_directory = Path.home() / "Documents/TSOSI/Fonts/Mont"

for file in otf_files:
    file_path_usefull = re.match(r".*(Mont[^\.]+)", file.name)
    save_path = (
        str(save_directory / file_path_usefull[1]).replace(" ", "_") + ".woff2"
    )
    compress(str(file), str(save_path))